<a href="https://colab.research.google.com/github/gomesGabriel/unip/blob/master/testes_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install opencv-contrib-python --upgrade

In [ ]:
from glob import glob
import cv2
import matplotlib.pyplot as plt
import os
import pytesseract
from matplotlib import pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow
from collections import Counter

In [ ]:
#Função que analisa letra que mais aparece dentro dos resultados
def retornaTexto(lista,repeticoes):
    caracteres = ''
    for i in range(repeticoes):
        letra = []
        for texto in lista:
            letra.append(texto[i]) #Pega a letra da mesma posição dos resultados e armazena em uma lista.
        letraStr = ''.join(map(str, letra)) #Transforma a lista em uma string.
        contador = Counter(letraStr) #Roda a função para gerar um dicionario contendo o caracter e a qtd de vezes que aparece.
        quantidade = 0
        for letra, qtd in contador.items(): #Estrutura que irá retornar o caracter que mais aparece.
            if qtd > quantidade:
                quantidade = qtd
                caracterOk = letra
        caracteres = caracteres + caracterOk
    return caracteres

In [ ]:
def analisaResultado(lista):
  #--------------------------------------------------------------------------------------------------------------#
  #------------------------------Inicia processo de analisar o resultado da leitura------------------------------#
  textoFinal = ''
  novalistaLetras = []
  novalistaNumeros = []
  #---------------------------------------------------------------------------------------------------------------
  for texto in lista:
      #Valida os caracteres e armazena só oque estiver dentro do padrão
      codigo = texto.replace(' ','')
      codigo = codigo.replace('\n\x0c','')
      texto = codigo[0:4]
      retorno = texto.isalpha()
      if(len(texto)<4):
        pass
      elif(retorno == True):
        novalistaLetras.append(texto)
      
      #Valida os números e armazena só oque estiver dentro do padrão
      texto = codigo[-7::]
      retorno = texto.isdigit()
      if(len(texto)<7):
        pass
      elif(retorno == True):
          novalistaNumeros.append(codigo[-7::])

  textoFinal = textoFinal + retornaTexto(novalistaLetras,4)
  textoFinal = textoFinal + retornaTexto(novalistaNumeros,7)

  return(textoFinal) 

In [ ]:
def aumentaResolucao(img):
  sr = cv2.dnn_superres.DnnSuperResImpl_create()
  path = "/content/EDSR_x4.pb"
  sr.readModel(path)
  sr.setModel("edsr",4)
  result = sr.upsample(img)
  # Resized image
  resized = cv2.resize(img,dsize=None,fx=4,fy=4)
  # plt.figure(figsize=(12,8))
  # plt.subplot(1,3,1)
  # # Original image
  # plt.imshow(img[:,:,::-1])
  # plt.subplot(1,3,2)
  # # SR upscaled
  # plt.imshow(result[:,:,::-1])
  # plt.subplot(1,3,3)
  # # OpenCV upscaled
  # plt.imshow(resized[:,:,::-1])
  # plt.show()
  return result

In [ ]:
def trocaCorLetra(newImg,x,y):
  for i in range(len(newImg[0:x,0:y])):
    for j in range(len(newImg[0:x,0:y][i])):
      if((newImg[0:x,0:y][i][j][0]) > 210 and (newImg[0:x,0:y][i][j][1]) > 210 and (newImg[0:x,0:y][i][j][2]) > 210): #Parâmetro que troca a cor do pixel para preto. (Quanto maior, mais fina fica a letra) 
        newImg[0:x,0:y][i][j][0] = 0
        newImg[0:x,0:y][i][j][1] = 0
        newImg[0:x,0:y][i][j][2] = 0
  return (newImg)

In [ ]:
def ocr(img,padrao,valor):
  imgOk = 0
  angulo = 0
  #Verificar se a imagem tem letra branca. Caso sim, troca pra preto.
  if(padrao == 0):
    x,y,pixels = np.shape(img)
    newImg = cv2.convertScaleAbs(img, alpha=1, beta=50) #Aumenta brilho e contraste
    newImg = trocaCorLetra(newImg,x,y)
    loopingThresh = 1
  else:
    newImg = img
    loopingThresh = 5
  
  cv2.imwrite(os.path.join('/content','ImagemNova.jpg'), newImg) #Coloca a nova imagem na pasta

  im_gray = cv2.imread('/content/ImagemNova.jpg', cv2.IMREAD_GRAYSCALE)
  altura, largura = im_gray.shape[:2]
  cv2.waitKey(0)
  ponto = (largura / 2, altura / 2) #ponto no centro da figura

  for i in range(4): #Looping referente a rotação da imagem
    valorThresh = valor
    lista = []
    for i in range(loopingThresh): #looping responsável por aumentar o valor de thresh
      rotacao = cv2.getRotationMatrix2D(ponto, angulo, 1.0) #Rotação da imagem para ajudar o tesseract a reconhecer melhor a imagem
      rotacionado = cv2.warpAffine(im_gray, rotacao, (largura, altura))
      cv2.waitKey(0)
      thresh = valorThresh 
      img = cv2.threshold(rotacionado, thresh, 255, cv2.THRESH_BINARY)[1]
      #---------------------------------------------------------------------------
      element_estr = cv2.getStructuringElement(cv2.MORPH_CROSS,(1,2)) #Ficou bom na matriz 1,2
      img_process = cv2.dilate(img, element_estr, iterations = 1)
      img_process2 = cv2.erode(img_process, element_estr, iterations = 2)
      
      cv2.imwrite(os.path.join('/content/Resultado','ImagemNova%04i.jpg' %i), img)
      if padrao == 0:
        valorThresh += 0
      else:
        valorThresh += 10


    #Esse looping analisa as imagens geradas do processamento de imagem usando o pytesseract
    for i in range(loopingThresh):
      config = r'--oem 3 --psm 6' 
      resultado = pytesseract.image_to_string('/content/Resultado/ImagemNova%04i.jpg' %i,config=config)
      #print(resultado)
      lista.append(resultado)

    try:
      texto = analisaResultado(lista)
      print(texto)
      imgOk += 1
    except Exception:
      print('Não encontrado')

    angulo -= 0.3
  return(imgOk)

In [ ]:
img=cv2.imread('/content/ctnr (16).jpg')
#img = aumentaResolucao(img)
resultado = ocr(img,0,1)
if(resultado == 0):
  ocr(img,1,30)